In [4]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

# Data

In [11]:
df = pd.read_csv("data/clean_data.csv")
df = df.apply(pd.to_numeric,errors = "coerce")
print(df.shape)

(7931, 9)


In [12]:
df = df.drop("date", axis=1)

In [13]:
x = df.values #returns a numpy array
scalar = preprocessing.MinMaxScaler()
x_scaled = scalar.fit_transform(x)
df = pd.DataFrame(x_scaled)
df.head(5)

,0,1,2,3,4,5,6,7
0,0.852144,0.808402,0.070671,0.020057,0.135459,0.408021,0.760988,0.342629
1,0.846501,0.809426,0.069336,0.020781,0.131714,0.411800,0.768947,0.346614
2,0.836343,0.800205,0.070078,0.021516,0.130130,0.413432,0.770729,0.352590
3,0.810384,0.781762,0.070671,0.024084,0.119474,0.425970,0.802804,0.393284
4,0.816027,0.786885,0.070449,0.023943,0.119778,0.422449,0.805417,0.416050


In [14]:
# x = df.values
# x_unscaled = scalar.inverse_transform(x)
# df = pd.DataFrame(x_unscaled)
# df.head(5)

In [15]:
train = np.array(df)

In [16]:
np.array(train).shape

(7931, 8)

# Model

In [25]:
import ESN

In [26]:
n_nodes = 10
dim_input = 8
W = ESN.RandomMatrix(n_nodes, 0, 0.9)
v = np.matrix(np.random.uniform(-1,1,size=n_nodes*dim_input)).reshape((n_nodes,dim_input))

In [27]:
def sigmoid(x, derivative=False):
    sigm = 1. / (1. + np.exp(-x))
    if derivative:
        return sigm * (1. - sigm)
    return sigm

In [74]:
nn_model = MNN()

In [75]:
# net = ESN.ESN(W,v,Activation=np.tanh)
net = ESN.ESN(W, v, Activation=np.tanh, alpha=1,beta=0,PCA=False,n_components=100,UseLinear=False,readout=nn_model)

In [76]:
x = Variable(torch.tensor(np.asarray(train[:-1])))
y = Variable(torch.tensor(np.asarray(train[1:])))

In [77]:
print(type(x))

<class 'torch.Tensor'>


In [78]:
net.fit(x, y)

/home/abelleulseged/miniconda3/envs/cs152/lib/python3.6/site-packages/ipykernel_launcher.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


loss:  0.08022651076316833


In [ ]:
true = df.tail(1)
pred = net.FreeRun(1)

# Analysis

In [ ]:
from sklearn.metrics import mean_squared_error as mse
mse(true, pred)

In [ ]:
pred = scalar.inverse_transform(pred)
true = scalar.inverse_transform(true)
mse(true, pred)

# Neural Network

In [73]:
import torch
import torch.nn as nn
from torch.autograd import Variable

class MNN:
    def __init__(self):
        self.n_in, self.n_h, self.n_out = 10, 7930, 8
        self.model = nn.Sequential(nn.Linear(self.n_in, self.n_h),
                         nn.ReLU(),
                         nn.Linear(self.n_h, self.n_out),
                         nn.Sigmoid())
        self.criterion = torch.nn.MSELoss()
        self.optimizer = torch.optim.SGD(self.model.parameters(), lr=0.01)
    
    def fit(self, inp, out):
        inp = torch.tensor(inp).float()
        out = torch.tensor(out).float()
        y_pred = self.model(inp)
        loss = self.criterion(y_pred, out)
        print('loss: ', loss.item())
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        
    def predict(self, inp):
        return self.model(inp)

In [ ]:
# Defining input size, hidden layer size, output size and batch size respectively
n_in, n_h, n_out = 30, 5, 8

In [ ]:
# Create dummy input and target tensors (data)
x = torch.randn(n_in)
y = torch.tensor([1,2,3,4,5,6,7,8])
y = y.float()

In [ ]:
# Create a model
model = nn.Sequential(nn.Linear(n_in, n_h),
                     nn.ReLU(),
                     nn.Linear(n_h, n_out),
                     nn.Sigmoid())

In [ ]:
# Construct the loss function
criterion = torch.nn.MSELoss()

# Construct the optimizer (Stochastic Gradient Descent in this case)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [ ]:
for epoch in range(50):
    # Forward pass: Compute predicted y by passing x to the model
    y_pred = model(x)

    # Compute and print loss
    loss = criterion(y_pred, y)
    print('epoch: ', epoch,' loss: ', loss.item())

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    
    # perform a backward pass (backpropagation)
    loss.backward()
    
    # Update the parameters
    optimizer.step()